# 字符级语言模型-恐龙之地

欢迎来到恐龙岛！ 6500万年前，恐龙就已经存在了，在这种任务下，它们又回来了。您负责一项特殊任务。领先的生物学研究人员正在创造新的恐龙品种，并将它们带入地球，而您的工作就是为这些恐龙起名字。如果恐龙不喜欢它的名字，它可能会发疯，所以请明智地选择！

<table>
<td>
<img src="images/dino.jpg" style="width:250;height:300px;">
​
</td>
​
</table>

幸运的是，您已经学习了一些深度学习，您将用它来节省时间。您的助手已收集了他们可以找到的所有恐龙名称的列表，并将其编译到此[dataset](dinos.txt)中。 （请单击上一个链接随意查看。）要创建新的恐龙名称，您将构建一个字符级语言模型来生成新名称。您的算法将学习不同的名称模式，并随机生成新名称。希望这种算法可以使您和您的团队免受恐龙的愤怒！

完成此作业，您将学到：

- 如何存储文本数据以使用RNN进行处理
- 如何通过在每个时间步采样预测并将其传递给下一个RNN单元来合成数据
- 如何建立字符级文本生成循环神经网络
- 为什么剪裁渐变很重要

我们将从加载`rnn_utils`中为您提供的一些功能开始。具体来说，您可以访问诸如`rnn_forward`和`rnn_backward`之类的功能，这些功能与您在上一个作业中实现的功能等效。

In [1]:
import numpy as np
from utils import *
import random
from random import shuffle

## 1-问题陈述

### 1.1-数据集和预处理

运行以下单元格以读取恐龙名称的数据集，创建唯一字符列表（例如a-z），并计算数据集和词汇量。

In [2]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


这些字符是a-z（26个字符）加上“ \ n”（或换行符），在此作业中，其作用类似于我们在讲座中讨论过的`<EOS>`（或“句子结尾”）标记， 仅在此处指示恐龙名称的结尾而不是句子的结尾。 在下面的单元格中，我们创建一个python字典（即哈希表），以将每个字符映射到0-26之间的索引。 我们还创建了第二个python字典，该字典将每个索引映射回对应的字符。 这将帮助您找出softmax层的概率分布输出中哪个索引对应于哪个字符。 下面，`char_to_ix`和`ix_to_char`是python字典。

In [5]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### 1.2 - 模型概述

您的模型将具有以下结构：

- 初始化参数
- 运行优化循环
    - 前向传播以计算损失函数
    - 后向传播以计算相对于损失函数的梯度
    - 修剪梯度，以避免爆炸梯度
    - 使用渐变，使用渐变下降更新规则更新参数。
- 返回学习的参数
    
<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center> **图1 **：递归神经网络，类似于您在上一个笔记本“构建RNN-逐步”中构建的内容。</center></caption>

在每个时间步长，RNN都会根据给定的先前字符来预测下一个字符。数据集$X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 是训练集中的字符列表，而$Y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ 使得每个时间步长$t$，我们有$y^{\langle t \rangle} = x^{\langle t+1 \rangle}$。

## 2 - 模型的构建块

在这一部分中，您将构建整个模型的两个重要模块：
- 渐变裁剪：避免渐变爆炸
- 采样：一种用于生成字符的技术

然后，您将应用这两个功能来构建模型。

### 2.1 - 在优化循环中剪切渐变

在本节中，您将实现在优化循环中调用的`clip`函数。回想一下，您的总体循环结构通常包括前向遍历，成本计算，后向遍历和参数更新。在更新参数之前，您将在需要时执行渐变剪切，以确保您的渐变不会“爆炸”，这意味着要采用太大的值。

在下面的练习中，您将实现一个函数`clip`，该函数接受渐变字典，并在需要时返回渐变的裁剪版本。裁剪渐变有多种方法。我们将使用简单的按元素裁剪程序，其中将梯度矢量的每个元素裁剪为位于某个范围 [-N, N]之间。通常，您将提供一个` maxValue`（例如10）。在此示例中，如果梯度向量的任何分量大于10，则将其设置为10；否则，将其设置为10。如果梯度向量的任何分量小于-10，则将其设置为-10。如果介于-10和10之间，则将其保留。

<img src="images/clip.png" style="width:400;height:150px;">
<caption><center> **图2 **：在网络遇到轻微的“爆炸梯度”问题的情况下，带有和不带有渐变剪切的渐变下降的可视化。 </center></caption>

**练习**：实现以下函数以返回字典`gradients`的剪切渐变。您的函数接受最大阈值，并返回渐变的剪切版本。您可以查看此[hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html)，以获取有关如何剪切numpy的示例。您将需要使用参数`out = ...`。

In [ ]:
### GRADED FUNCTION: clip

def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    ### START CODE HERE ###
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in [dWax, dWaa, dWya, db, dby]:
        None
    ### END CODE HERE ###
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [ ]:
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, 10)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

**预期输出:**

<table>
<tr>
    <td> 
    **gradients["dWaa"][1][2] **
    </td>
    <td> 
    10.0
    </td>
</tr>

<tr>
    <td> 
    **gradients["dWax"][3][1]**
    </td>
    <td> 
    -10.0
    </td>
    </td>
</tr>
<tr>
    <td> 
    **gradients["dWya"][1][2]**
    </td>
    <td> 
0.29713815361
    </td>
</tr>
<tr>
    <td> 
    **gradients["db"][4]**
    </td>
    <td> 
[ 10.]
    </td>
</tr>
<tr>
    <td> 
    **gradients["dby"][1]**
    </td>
    <td> 
[ 8.45833407]
    </td>
</tr>

</table>

### 2.2 - 采样

现在假设您的模型已经过训练。您想生成新文本（字符）。下图说明了生成过程：

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center>**图3 **：在此图中，我们假设模型已经过训练。我们在第一步中传入 $x^{\langle 1\rangle} = \vec{0}$ ，然后让网络一次采样一个字符。</center></caption>

**练习**：实施以下的“采样”功能来采样字符。您需要执行4个步骤：

- **步骤1**：将第一个“虚拟”输入$x^{\langle 1 \rangle} = \vec{0}$（零向量）传递给网络。这是我们生成任何字符之前的默认输入。我们还设置$a^{\langle 0 \rangle} = \vec{0}$

- **步骤2**：执行向前传播的步骤，即可获得$a^{\langle 1 \rangle}$ 和 $\hat{y}^{\langle 1 \rangle}$。以下是等式：

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

注意 $\hat{y}^{\langle t+1 \rangle }$是一个（softmax）概率向量（其条目在0到1之间且总和为1）。 $\hat{y}^{\langle t+1 \rangle}_i$表示由“ i”索引的字符是下一个字符的概率。我们提供了一个`softmax（）`函数供您使用。

- **步骤3**：执行抽样：根据$\hat{y}^{\langle t+1 \rangle }$指定的概率分布，选择下一个字符的索引。这意味着，如果$\hat{y}^{\langle t+1 \rangle }_i = 0.16$，您将以16％的概率选择索引“ i”。要实现它，您可以使用 [`np.random.choice`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)。

这是一个如何使用`np.random.choice（）`的例子：
```python
np.random.seed(0)
p = np.array([0.1, 0.0, 0.7, 0.2])
index = np.random.choice([0, 1, 2, 3], p = p.ravel())
```
这意味着您将根据分布选择`index`：
P（索引=0）=0.1，P（索引=1）=0.0，P（索引=2）=0.7，P（索引=3）=0.2P（索引=0）=0.1，P（索引=1）=0.0，P（索引=2）=0.7，P（索引=3）=0.2。

- **步骤4**：要在`sample（）`中实现的最后一步是覆盖变量`x`，该变量当前存储$x^{\langle t \rangle }$，其值为$x^{\langle t + 1 \rangle }$。通过创建与您选择作为预测的字符相对应的单热点矢量，您将表示$x^{\langle t + 1 \rangle }$。然后，您将在步骤1中向前传播$x^{\langle t + 1 \rangle }$，并继续重复该过程，直到获得“ \ n”字符，表明您已到达恐龙名称的末尾。

In [ ]:
# GRADED FUNCTION: sample

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = None
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = None
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevents entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = None
        z = None
        y = None
        
        # for grading purposes
        np.random.seed(counter+seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = None

        # Append the index to "indices"
        None
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        x = None
        x[None] = None
        
        # Update "a_prev" to be "a"
        a_prev = None
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [ ]:
np.random.seed(2)
n, n_a = 20, 100
a0 = np.random.randn(n_a, 1)
i0 = 1 # first character is ix_to_char[i0]
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:", indices)
print("list of sampled characters:", [ix_to_char[i] for i in indices])

** 预期输出:**
<table>
<tr>
    <td> 
    **list of sampled indices:**
    </td>
    <td> 
    [18, 2, 26, 0]
    </td>
    </tr><tr>
    <td> 
    **list of sampled characters:**
    </td>
    <td> 
    ['r', 'b', 'z', '\n']
    </td>
    
        
    
</tr>
</table>

## 3 - 建立语言模型

现在是时候建立用于文字生成的字符级语言模型了。


### 3.1-梯度下降

在本节中，您将实现一个函数，该函数执行随机梯度下降的一个步骤（带有修剪的梯度）。您将一次查看一个训练示例，因此优化算法将是随机梯度下降。提醒一下，这是RNN常见优化循环的步骤：

- 通过RNN向前传播以计算损耗
- 随时间向后传播以计算相对于参数的损耗梯度
- 必要时剪切渐变
- 使用梯度下降更新参数

**练习**：实施此优化过程（随机梯度下降的一个步骤）。

我们为您提供以下功能：

```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in the backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```

In [ ]:
# GRADED FUNCTION: optimize

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    ### START CODE HERE ###
    
    # Forward propagate through time (≈1 line)
    loss, cache = None
    
    # Backpropagate through time (≈1 line)
    gradients, a = None
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = None
    
    # Update parameters (≈1 line)
    parameters = None
    
    ### END CODE HERE ###
    
    return loss, gradients, a[len(X)-1]

In [ ]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

** 预期输出:**

<table>


<tr>
    <td> 
    **Loss **
    </td>
    <td> 
    126.503975722
    </td>
</tr>
<tr>
    <td> 
    **gradients["dWaa"][1][2]**
    </td>
    <td> 
    0.194709315347
    </td>
<tr>
    <td> 
    **np.argmax(gradients["dWax"])**
    </td>
    <td> 93
    </td>
</tr>
<tr>
    <td> 
    **gradients["dWya"][1][2]**
    </td>
    <td> -0.007773876032
    </td>
</tr>
<tr>
    <td> 
    **gradients["db"][4]**
    </td>
    <td> [-0.06809825]
    </td>
</tr>
<tr>
    <td> 
    **gradients["dby"][1]**
    </td>
    <td>[ 0.01538192]
    </td>
</tr>
<tr>
    <td> 
    **a_last[4]**
    </td>
    <td> [-1.]
    </td>
</tr>

</table>

### 3.2 - 训练模型

给定恐龙名称数据集，我们将数据集的每一行（一个名称）用作一个训练示例。每100步随机梯度下降，您将采样10个随机选择的名称，以查看算法的运行情况。请记住要对数据集进行混洗，以便随机梯度下降以随机顺序访问示例。

**练习**：按照说明进行操作并实现`model（）`。当`examples [index]`包含一个恐龙名称（字符串）以创建示例（X，Y）时，可以使用以下方法：
```python
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
```
请注意，我们使用：`index= j % len(examples)`，其中`j = 1....num_iterations`，以确保`examples[index]`始终是有效语句（`index`小于`len(examples)`。
`X`的第一个条目为`None`将被`rnn_forward()`解释为设置$x^{\langle 0 \rangle} = \vec{0}$。此外，这可以确保 `Y`等于`X`，但向左移动了一步，并附加了附加的“ \ n”以表示恐龙名称的结尾。

In [ ]:
# GRADED FUNCTION: model

def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text, size of the vocabulary
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Shuffle list of all dinosaur names
    shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        ### START CODE HERE ###
        
        # Use the hint above to define one training example (X,Y) (≈ 2 lines)
        index = None
        X = None
        Y = None
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = None
        
        ### END CODE HERE ###
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result for grading purposed, increment the seed by one. 
      
            print('\n')
        
    return parameters

运行以下单元格，您应该观察到模型在第一次迭代时输出看起来随机的字符。 经过数千次迭代后，您的模型应该学会生成看起来合理的名称。

In [ ]:
parameters = model(data, ix_to_char, char_to_ix)

## 结论

您可以看到，在训练即将结束时，您的算法已开始生成合理的恐龙名称。刚开始时，它会生成随机字符，但是到最后，您会看到恐龙名字的结尾很酷。随意运行该算法甚至更长，并使用超参数来看看是否可以获得更好的结果。我们的实现产生了一些非常酷的名称，例如`maconucon`, `marloralus`和`macingsersaurus`。您的模型还有望了解到恐龙名称往往以`saurus`，`don`，`aura`，`tor`等结尾。

如果您的模型生成了一些不酷的名字，请不要完全怪罪模型-并非所有实际的恐龙名字听起来都很酷。 （例如，`dromaeosauroides`是实际的恐龙名称，并且在训练集中。）但是此模型应该为您提供一组候选人，从中可以挑选最酷的候选人！

该分配使用了相对较小的数据集，因此您可以在CPU上快速训练RNN。训练英语模型需要更大的数据集，并且通常需要更多的计算量，并且可以在GPU上运行多个小时。我们使用恐龙的名字已经有一段时间了，到目前为止，我们最喜欢的名字是伟大的，不败的和凶猛的：芒果龙！
<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">

## 4-像莎士比亚一样写作

该笔记本的其余部分是可选的，尚未评分，但我们希望您无论如何都可以做，因为它非常有趣且内容丰富。

一个类似（但更复杂）的任务是生成莎士比亚诗歌。 无需从恐龙名称的数据集中学习，而可以使用莎士比亚诗歌集。 使用LSTM单元，您可以学习跨文本中许多字符的长期依赖关系-例如，某个字符出现在序列的某个地方可能会影响序列后面的其他字符。 这些长期依赖关系对于恐龙名称来说并不重要，因为它们的名称很短。


<img src="images/shakespeare.jpg" style="width:500;height:400px;">
<caption><center> Let's become poets! </center></caption>

我们已经与Keras一起实现了莎士比亚诗歌生成器。 运行以下单元格以加载所需的软件包和模型。 这可能需要几分钟的时间。

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from shakespeare_utils import *
import sys
import io

为了节省您的时间，我们已经在名为 [*"The Sonnets"*](shakespeare.txt)的莎士比亚诗歌集上训练了大约1000个时期的模型。

让我们再训练一个模型。 完成训练一个新纪元后-这也将花费几分钟-您可以运行`generate_output`，这将提示您要求输入（`<`40个字符）。 这首诗将从您的句子开始，而我们的RNN莎士比亚将为您完成其余的诗！ 例如，尝试“无意义的解决”（不要输入引号）。 根据您是否在结尾处加上空格，您的结果也可能会有所不同-双向尝试，也尝试其他输入。

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

In [ ]:
# Run this cell to try with different inputs without having to re-train the model 
generate_output()

RNN-莎士比亚模型与您为恐龙名称建立的模型非常相似。 唯一的主要区别是：
- LSTM代替基本的RNN以捕获更远的依赖
- 模型是更深的堆叠LSTM模型（2层）
- 使用Keras而不是python来简化代码

如果您想了解更多信息，还可以在GitHub上查看Keras Team的文本生成实现https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py.
祝贺您完成本笔记本！

**参考**：
- 此练习从Andrej Karpathy的实现中获得启发： https://gist.github.com/karpathy/d4dee566867f8291f086。 要了解有关文本生成的更多信息，请查看Karpathy的[blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)。
- 对于莎士比亚诗歌生成器，我们的实现基于Keras团队对LSTM文本生成器的实现：https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py 